In [ ]:
# Model comparisons

In [1]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.datasets.dataset_300W import Dataset300W
from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NME_mean
from loreal_poc.tests.base import Test, TestDiff

In [2]:
ds = Dataset300W(dir_path="300W/sample")
chosen_idx = 4
image = ds.all_images[chosen_idx]
ground_truth_landmarks = ds.all_marks[chosen_idx, :, :]

In [3]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False))
opencv_model = OpenCVWrapper()

File exists
File exists
loading data from : lbfmodel.yaml


In [17]:
Test(metric=NME_mean, threshold=1).run(model=opencv_model, dataset=ds)

Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.0414 (threshold = 1)
               
               Prediction time: 0.48 s.

In [5]:
Test(metric=NME_mean, threshold=1).run(model=facealignment_model, dataset=ds)

Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.0623 (threshold = 1)
               
               Prediction time: 37.83 s.

In [8]:
from loreal_poc.transformation_functions.transformation_functions import crop_based_on_facial_part
from loreal_poc.datasets.base import FacialParts

facial_part = FacialParts.left_half

cropped_ds = crop_based_on_facial_part(ds, facial_part, rel_margin=0.00)

TestDiff(metric=NME_mean, threshold=1).run(
    model=opencv_model,
    dataset=ds,
    transformation_function=crop_based_on_facial_part,
    transformation_function_kwargs={"dataset": ds, "facial_part": facial_part, "rel_margin": 0.00},
)

Test "NME_mean" succeeded
               Description: Mean of normalised mean Euclidean distances across images
               Metric: 0.7037 (threshold = 1)
               Prediction fail rate: 0.4
               Prediction time: 0.54 s.
               Data preprocessing time: 0.15 s.

In [24]:
from dataclasses import fields

results = {"OpenCV": {"model": opencv_model}, "FaceAlignment": {"model": facealignment_model}}
for model_name, _model in results.items():
    model = _model["model"]
    for __facial_part in fields(FacialParts):
        _facial_part = __facial_part.default
        kwargs = {"dataset": ds, "facial_part": _facial_part, "rel_margin": 0.00}
        cropped_ds = crop_based_on_facial_part(ds, _facial_part, rel_margin=0.00)
        test = TestDiff(metric=NME_mean, threshold=1).run(
            model=model,
            dataset=ds,
            transformation_function=crop_based_on_facial_part,
            transformation_function_kwargs=kwargs,
        )
        results[model_name][_facial_part.name] = test.metric

/Users/rak/Documents/loreal-poc/.venv/lib/python3.11/site-packages/face_alignment/api.py:147: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


In [25]:
results

{'OpenCV': {'model': <loreal_poc.models.wrappers.OpenCVWrapper at 0x107a092d0>,
  'entire face': 0.031593074968095824,
  'face contour': 0.9622098511041972,
  'left eyebrow': nan,
  'right eyebrow': nan,
  'nose': nan,
  'left eye': nan,
  'right eye': nan,
  'mouth': nan,
  'bottom half': nan,
  'upper half': 0.12371467791790752,
  'left half': 0.7037441359556575,
  'right half': 0.6868418487365847},
 'FaceAlignment': {'model': <loreal_poc.models.wrappers.FaceAlignmentWrapper at 0x161415e90>,
  'entire face': 0.06824182183674955,
  'face contour': 0.662105542984795,
  'left eyebrow': nan,
  'right eyebrow': nan,
  'nose': nan,
  'left eye': 0.607967679637122,
  'right eye': nan,
  'mouth': nan,
  'bottom half': 0.09077343744617554,
  'upper half': 0.1243827658614352,
  'left half': 0.641158345036311,
  'right half': 0.30052643001034707}}